# GeoRoute LSTM Evaluation

In this notebook we run the test dataaset through the model and compute the accuracy.

In [1]:
import numpy as np
import torch
from models.rnn import CellType
from models.geo_route_lstm import GeoRouteLSTM
from torch.utils.data import Dataset, DataLoader
from dataset.geo_route import GeoRouteDataset, prepare_tensors

%load_ext autoreload
%autoreload 2

In [2]:
# Find out if a CUDA device (GPU) is available
if torch.cuda.device_count():
    device="cuda"
else:
    device="cpu"
print("Device", device)

Device cpu


c:\Users\tilof\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\cuda\__init__.py:740: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ..\c10\cuda\CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


In [3]:
# File that contains the data
dataset_file = "dataset_tiny.pkl.gz"
model_dict_file = "geo_route_model.pt"


model_dict = torch.load(model_dict_file, map_location=torch.device(device))

# Cell type (LSTM | GRU | RNN)
cell_type=model_dict["cell_type"]
# Number of RNN layers
num_layers=model_dict["num_layers"]
# Embedding dimension
embedding_dim=model_dict["embedding_dim"]
# Hidden size of the RNN layers
hidden_size=model_dict["hidden_size"]
# Batch size used for training
batch_size=1024
# Maximum sequence length
max_length=model_dict["max_length"]
# True if bidirectional RNN layers should be used, False otherwise
bidirectional=model_dict["bidirectional"]

In [4]:
# Create an instance of the dataset and a dataloader
dataset = GeoRouteDataset(dataset_file)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [5]:
# Instantiate the network
net = GeoRouteLSTM(device=device).to(device)
net.load_state_dict(model_dict["net_state_dict"])

<All keys matched successfully>

In [6]:
# Switch to evaluation mode
net.eval()

GeoRouteLSTM(
  (embedding): GeoRouteEmbedding(
    (embedding_src_as): Embedding(5, 2)
    (embedding_dest_as): Embedding(52230, 16)
    (embedding_src_cc): Embedding(4, 2)
    (embedding_dest_cc): Embedding(231, 8)
    (embedding_ip_source): Embedding(5, 3)
    (embedding_geo_cc): Embedding(252, 8)
    (embedding_asn): Embedding(397771, 19)
  )
  (encoder): RecurrentEncoder(
    (rnn): LSTM(32, 256, num_layers=3, batch_first=True, dropout=0.1, bidirectional=True)
  )
  (classifier): Classifier(
    (fc1): Linear(in_features=540, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=2, bias=True)
    (softmax): LogSoftmax(dim=2)
  )
)

In [7]:
n_class_0_all = 0
n_class_1_all = 0
accuracies_class_0 = []
accuracies_class_1 = []

# Get a batch of training data
for src_as, dest_as, src_cc, dest_cc, lat, long, asn, ip_source, geo_cc, labels in dataloader:
    src_as, dest_as, src_cc, dest_cc, lat, long, asn, ip_source, geo_cc, labels = prepare_tensors(
        src_as, dest_as, src_cc, dest_cc, lat, long, asn, ip_source, geo_cc, labels, device=device
    )

    # Turn labels into torch.long
    labels = labels.to(torch.long).to(device)
        
    # Create masks for positive and negative labels
    mask_class_0 = labels.squeeze() == 0
    mask_class_1 = labels.squeeze() == 1
        
    # Get logits for each of the two classes
    logits = net(
        lat=lat,
        long=long,
        asn=asn,
        ip_source=ip_source,
        geo_cc=geo_cc,
        src_as=src_as,
        dest_as=dest_as,
        src_cc=src_cc,
        dest_cc=dest_cc,
    )
        
    # Get the most likely class for each input
    topv, topi = logits.topk(1)
        
    # Get number of positive and neagtive samples
    n_class_0 = mask_class_0.sum().item()
    n_class_1 = mask_class_1.sum().item()

    # Update number of total items
    n_class_0_all += n_class_0
    n_class_1_all += n_class_1
        
    # Compute total accuracy and accuracies for both positive and negative samples
    matchings = labels.squeeze() == topi.squeeze()
    accuracy_total = matchings.sum().item() / batch_size
    accuracy_class_0 = matchings[mask_class_0].sum().item() / n_class_0 if n_class_0 > 0 else 0.0
    accuracy_class_1 = matchings[mask_class_1].sum().item() / n_class_1 if n_class_1 > 0 else 0.0

    # Save accuracies
    accuracies_class_0.append(accuracy_class_0)
    accuracies_class_1.append(accuracy_class_1)
        
    print(f"AccAll", round(accuracy_total, 3), "Acc0", round(accuracy_class_0, 3), "Acc1", round(accuracy_class_1, 3))

AccAll 0.939 Acc0 0.944 Acc1 0.848
AccAll 0.944 Acc0 0.952 Acc1 0.73
AccAll 0.95 Acc0 0.954 Acc1 0.88
AccAll 0.946 Acc0 0.951 Acc1 0.848
AccAll 0.94 Acc0 0.95 Acc1 0.769
AccAll 0.942 Acc0 0.957 Acc1 0.66
AccAll 0.941 Acc0 0.953 Acc1 0.725
AccAll 0.958 Acc0 0.963 Acc1 0.875
AccAll 0.947 Acc0 0.954 Acc1 0.8
AccAll 0.936 Acc0 0.944 Acc1 0.761
AccAll 0.951 Acc0 0.957 Acc1 0.852
AccAll 0.939 Acc0 0.948 Acc1 0.771
AccAll 0.942 Acc0 0.949 Acc1 0.83
AccAll 0.947 Acc0 0.961 Acc1 0.738
AccAll 0.952 Acc0 0.962 Acc1 0.739
AccAll 0.943 Acc0 0.952 Acc1 0.8
AccAll 0.95 Acc0 0.957 Acc1 0.795
AccAll 0.939 Acc0 0.948 Acc1 0.676
AccAll 0.951 Acc0 0.958 Acc1 0.791
AccAll 0.95 Acc0 0.956 Acc1 0.852
AccAll 0.941 Acc0 0.946 Acc1 0.862
AccAll 0.951 Acc0 0.956 Acc1 0.806
AccAll 0.952 Acc0 0.956 Acc1 0.87
AccAll 0.95 Acc0 0.954 Acc1 0.857
AccAll 0.936 Acc0 0.948 Acc1 0.738
AccAll 0.945 Acc0 0.949 Acc1 0.877
AccAll 0.953 Acc0 0.962 Acc1 0.776
AccAll 0.938 Acc0 0.944 Acc1 0.833
AccAll 0.948 Acc0 0.956 Acc1 0.778


In [10]:
mean_acc_class_0 = np.array(accuracies_class_0).mean()
mean_acc_class_1 = np.array(accuracies_class_1).mean()

print(f"Accuracy class 0: {mean_acc_class_0}, Accuracy class 1: {mean_acc_class_1}")

Accuracy class 0: 0.9524723764559416, Accuracy class 1: 0.8043173489232932
